# Test script

Script to run a test on recent data. The underlying assumption of the testing procedure is that if an ML model predicts that a drive will fail, then the hard drive would be taken out of operation. Therefore, at each day predictions are only made for drives for which no prediction of failure had been made prior to
that day.

The primary metrics we wish to track are the number of unexpected breaks and the number of false alarms that would have been output by the ML models had they been used during the first half of 2019.

In [1]:
import os
import glob

import numpy as np
import pandas as pd

import pickle

from sklearn import metrics
import xgboost as xgb

In [10]:
BASELINE_FEATURES = ['smart_5_raw', 'smart_187_raw',
                     'smart_188_raw', 'smart_197_raw', 'smart_198_raw']

# MODELS = ['baseline',
#           'fail_today_xgb',
#           'fail_today_or_tomorrow_xgb',
#           'fail_this_week_xgb',
#           'fail_this_month_xgb',
#           'fail_today_xgb_history',
#           'fail_today_or_tomorrow_xgb_history',
#           'fail_this_week_xgb_history',
#           'fail_this_month_xgb_history']
MODELS = ['fail_today_forest',
          'fail_today_or_tomorrow_forest',
          'fail_this_week_forest',
          'fail_this_month_forest',
          'fail_today_forest_history',
          'fail_today_or_tomorrow_forest_history',
          'fail_this_week_forest_history',
          'fail_this_month_forest_history']

HDD_PATH = os.path.join('datasets', 'drive_stats', 'test')

# Change these if needed!
MODEL_PATH = os.path.join('exploring_models')

In [11]:
def get_test_data(test_location):
    """
    Function to get all test data 
    
    Parameters
    ----------
    test_location:
        Where test csv's are located
        
    Returns
    -------
    concatenated_df : DataFrame
        A dataframe containing all of the data from the csvs
    """
    # List the relevant files in the bucket
    files = glob.glob(os.path.join(test_location, '*.csv'))
    # Read the files, concatenate them, and sort by date
    # df_for_each_blob = (pd.read_csv(file) for file in files)
    concatenated_df = pd.concat(pd.read_csv(file) for file in files).sort_values('date')
    # Some drives failed between Jan 1st and Jan 10th, we remove them here
    # (This could also just have been done in BQ ahead of time)
    already_removed_drives = concatenated_df[(concatenated_df['failure'] == 1) 
                                             & (concatenated_df['date'] < '2019-01-10')]['serial_number'].values
    concatenated_df = concatenated_df[~concatenated_df['serial_number'].isin(already_removed_drives)]
    cols_to_drop = [col for col in concatenated_df.columns if 'normalized' in col]
    concatenated_df.drop(columns=cols_to_drop, inplace=True)
    concatenated_df
    
    return concatenated_df

In [12]:
def get_model(model, model_path):
    """
    Load a saved model so we can use it for predictions
    
    Parameters
    ----------
    model : string
        Name of the model
        
    model_path : string
        Path to the saved model
        
    Returns
    -------
    loaded_model : Model
        Trained Sklearn/xgboost model used to make predictions
    """
    model_file = os.path.join(model_path, model + '.pickle.dat')
    loaded_model = pickle.load(open(model_file, "rb"))
    
    return loaded_model

In [15]:
def run_test(model, test_data, model_path):
    """
    Run a test for a particular model

    Parameters
    ----------
    model : string
        Name of the model

    test_data : DataFrame
        Data for which we'd like to run the test

    model_path :

    Returns
    -------
    results : dict
    """
    date_list = test_data['date'].unique()

    # Load model and figure out which aggregations to do
    if model != 'baseline':
        loaded_model = get_model(model, model_path)

    # Same aggregations we used for training
    if 'history' in model:
        aggs = ['last', np.mean, np.var, 'min', 'max']
    else:
        aggs = ['last']

    # Keep track of the drives we have "removed".
    # These represent drives which have failed or that we have
    # done maintenance on.
    removed_drives = set()
    # The two primary metrics we care about
    unexpected_breaks = 0
    false_alarms = 0

    print('{:=^80}'.format(' Evaluating {} '.format(model)))

    # Now basically do 10 day sliding windows, where 10
    # is the length of our historical window.
    for i in range(len(date_list) - 9):
        dates = date_list[i:i+10]

        # Get 10 days of history for drives that have not been 'removed'
        window_df = test_data[(test_data['date'].isin(dates)) & ~(
            test_data['serial_number'].isin(removed_drives))]

        grouped_df = window_df.groupby('serial_number')
        features = grouped_df[[
            col for col in window_df.columns if 'smart' in col]]
        features = features.agg(aggs)
        # Fill nulls
        for col in features:
            features[col] = features[col].fillna(features[col].mode()[0])

        results = grouped_df['failure'].agg(['last'])

        if model == 'baseline':
            results['prediction'] = np.any(features[BASELINE_FEATURES].values >
                                           0, axis=1).astype(int)
        else:
            results['prediction'] = loaded_model.predict(features)

        predicted_failed_drives = set(
            results[results['prediction'] == 1].index)
        actual_failed_drives = set(results[results['last'] == 1].index)

        # Actually failed but not predicted
        unexpected_breaks += len(actual_failed_drives
                                 - predicted_failed_drives)
        # Predicted but didn't fail
        false_alarms += len(predicted_failed_drives - actual_failed_drives)
        # Both predicted and actually failed are removed
        removed_drives.update(
            predicted_failed_drives.union(actual_failed_drives))

        # There are a lot of false alarms on the first day, keep track of
        # this number
        if i == 0:
            initial_false_alarms = false_alarms

        if i % 20 == 0:
            print("By {} we have {} unexpected breaks and {} false alarms"
                  .format(dates[-1],
                          unexpected_breaks,
                          false_alarms))

    final_results = {'Unexpected Breaks': unexpected_breaks,
                     'False Alarms': false_alarms,
                     'Initial False Alarms': initial_false_alarms}

    print(final_results)
    print('{:-^80}'.format(''))

    return final_results

In [17]:
test_data = get_test_data(HDD_PATH)

In [18]:
# Location of saved models on disk
model_path = MODEL_PATH

results = {}
for model in MODELS:
    results[model] = run_test(model, test_data, model_path)

results_frame = pd.DataFrame(results).T
results_frame.to_csv('test_results_forest.csv')

========================= Evaluating fail_today_forest =========================
By 2019-01-10 we have 1 unexpected breaks and 968 false alarms
By 2019-01-30 we have 8 unexpected breaks and 1489 false alarms
By 2019-02-19 we have 19 unexpected breaks and 1698 false alarms
By 2019-03-11 we have 21 unexpected breaks and 1798 false alarms
By 2019-03-31 we have 27 unexpected breaks and 1853 false alarms
By 2019-04-20 we have 34 unexpected breaks and 1903 false alarms
By 2019-05-10 we have 45 unexpected breaks and 1984 false alarms
By 2019-05-30 we have 46 unexpected breaks and 2035 false alarms
{'Unexpected Breaks': 46, 'False Alarms': 2037, 'Initial False Alarms': 968}
--------------------------------------------------------------------------------
=================== Evaluating fail_today_or_tomorrow_forest ===================
By 2019-01-10 we have 1 unexpected breaks and 952 false alarms
By 2019-01-30 we have 9 unexpected breaks and 1363 false alarms
By 2019-02-19 we have 19 unexpected 